In [1]:
import pandas as pd
from google.cloud import bigquery

client = bigquery.Client.from_service_account_json('google_token.json')

In [2]:
# Download query results.
query_string = """
with tmp as (
select
user_pseudo_id
, datetime(timestamp_micros(event_timestamp), "Asia/Bangkok") as event_time
, datetime(timestamp_micros(user_first_touch_timestamp), "Asia/Bangkok") as first_time
, t2.value.string_value as screen

FROM `taptap-c2c5f.analytics_215493450.events_*` t1
cross join unnest(event_params) t2
where 1=1
and event_name = 'screen_view'
and t2.key = "firebase_screen"
--and upper(t2.value.string_value) not like "%SIGNUP%"
and _table_suffix >= "20220201"
--and _table_suffix < "20220301"
)
, tmp1 as (
select tmp.*
, lag(event_time) over (partition by user_pseudo_id order by event_time) as previous_time
from tmp
)
, tmp2 as (
select * 
, datetime_diff(event_time, previous_time, second) as _diff
, if(datetime_diff(event_time, previous_time, second) is null or datetime_diff(event_time, previous_time, second) > 300, 1, 0) as check_session
from tmp1
)
, tmp3 as (
select *
, sum(check_session) over (partition by user_pseudo_id order by event_time) as session
from tmp2
)
, tmp4 as (
select *
, lead(event_time) over (partition by user_pseudo_id, session order by event_time) as next_time
from tmp3
)
, tmp5 as (
select *
, datetime_diff(next_time, event_time, second) as view_time
from tmp4
)
, tmp6 as (
select * from tmp5
where 1=1
and date(event_time) = date(first_time)
-- and session = 1
)
, re as (
select user_pseudo_id, min(event_time) as retain_time
from tmp5
where 1=1
and session > 1
and date(event_time) > date(first_time)
group by 1
)
, tmp7 as (
select distinct
t1.*, t2.retain_time
, min(event_time) over(partition by t1.user_pseudo_id) as min_event_time
from tmp6 t1
left join re as t2 on t1.user_pseudo_id = t2.user_pseudo_id
)
, tmp8 as (
select *
, datetime_diff(retain_time, min_event_time, minute) as gap
, if(retain_time is null, 0, if(datetime_diff(retain_time, min_event_time, hour)>24,1,1)) as type
from tmp7
)
, sob as (

    with tmp as (
    select distinct t1.user_pseudo_id, t1.string_value as phone
    , case when t2.user_pseudo_id is not null then "outlier_pseudo" else ifnull(t3.name, "first_lc") end as sob
    from `taptap-c2c5f.manual_dwh.report_mapping_pseudo_id` as t1
    left join (
        SELECT 
        user_pseudo_id
        , count(distinct string_value)
        FROM `taptap-c2c5f.manual_dwh.report_mapping_pseudo_id` 
        group by 1
        having count(distinct string_value) > 1
    ) as t2 on t1.user_pseudo_id = t2.user_pseudo_id
    left join (
        SELECT  t2.name, t1.phoneNumber
        FROM `taptap-c2c5f.manual_dwh.ram_sign_on_bonus__CustomerPackage_*` t1
        left join `taptap-c2c5f.manual_dwh.ram_sign_on_bonus__Campaign_*` t2 on t1.campaignId = json_extract_scalar(replace(t2._id,"$",""), "$.oid")
        where 1=1
        and t1.state = 'SUCCESSFUL'
    ) as t3 on t1.string_value = t3.phoneNumber
    )
    select distinct t1.user_pseudo_id, if(t2.phone is null, t1.sob, "outlier_phone") sob
    from tmp t1
    left join (
        SELECT 
        string_value as phone
        , count(distinct user_pseudo_id)
        FROM `taptap-c2c5f.manual_dwh.report_mapping_pseudo_id` 
        group by 1
        having count(distinct user_pseudo_id) > 1
    ) as t2 on t1.phone = t2.phone
)
, txn as (
    with tmp as (
        select distinct t1.user_pseudo_id
        , t1.string_value as phone, first_date
        -- , if(t5 is not null, 1,0) as check_txn
        from `taptap-c2c5f.manual_dwh.report_mapping_pseudo_id` as t1
        left join (
            SELECT 
            user_pseudo_id
            , count(distinct string_value)
            FROM `taptap-c2c5f.manual_dwh.report_mapping_pseudo_id` 
            group by 1
            having count(distinct string_value) > 1
        ) as t2 on t1.user_pseudo_id = t2.user_pseudo_id
        left join (
            SELECT 
            string_value as phone
            , count(distinct user_pseudo_id)
            FROM `taptap-c2c5f.manual_dwh.report_mapping_pseudo_id` 
            group by 1
            having count(distinct user_pseudo_id) > 1
        ) as t3 on t1.string_value = t3.phone
        inner join (
            select user_pseudo_id
            , min(date(timestamp_micros(user_first_touch_timestamp), "Asia/Bangkok")) as first_date
            from `taptap-c2c5f.analytics_215493450.events_*`
            group by 1
        ) t4 on t1.user_pseudo_id = t4.user_pseudo_id
        where 1=1
        and t2.user_pseudo_id is null
        and t3.phone is null
    )

    select t1.user_pseudo_id, if(t5.phone is null, 0, 1) as check_txn
    from tmp as t1
    left join (
        select distinct phone, transaction_date
        from `taptap-c2c5f.View_Unified.combine_transaction`
    ) t5 on t1.phone = t5.phone and t1.first_date = t5.transaction_date 
)
-------------------------------------------------------------------
, tmp9 as (
select t1.user_pseudo_id
, type
, t2.sob
, t3.check_txn
--, count(distinct session) as no_session
, count(*)-1 as no_screen
, count(distinct screen) as no_unique_screen
, sum(view_time) as session_time
, max(view_time) as max_view_time
, max(if(next_time is null, screen, null)) as last_screen
, sum(if(screen in ("HOME_TAB", "home"), view_time, 0)) as home_a
, sum(if(screen in ("ACCOUNT_TAB", "myAccount", "MY_INFO", "INPUT_CODE", "HISTORY"), view_time, 0)) as account
, sum(if(screen in ("REWARD_TAB", "VOUCHER_DETAIL"), view_time, 0)) voucher
, sum(if(screen in ("MY_VOUCHER_LIST", "MY_VOUCHER_DETAIL"), view_time, 0)) my_voucher
, sum(if(screen in ("OCR_TAB", "OCR_BILL_LIST", "earn_function"), view_time, 0)) OCR
--, sum(if(screen in ("PAYMENT_TAB"), view_time, 0)) payment
, sum(if(screen in ("BRAND_INFO", "LIST_BRANDS", 'merchantDetail', "LIST_VOUCHERS_BRAND"), view_time, 0)) brand
, sum(if(screen in ("AFFLIATE_BRAND_LIST"), view_time, 0)) affiliate
, sum(if(screen in ("HOT_NEWS_BANNER_DETAIL"), view_time, 0)) news
, sum(if(screen in ("AVATAR_BOARD"), view_time, 0)) avatar
, sum(if(screen in ("INBOX"), view_time, 0)) inbox
, sum(if(screen in ("MY_CHALLENGES"), view_time, 0)) challenge
, sum(if(screen in ("WHEEL_FOR_TUNE"), view_time, 0)) wof
, count(distinct session) as no_session
--, sum(if(screen in ("WHEEL_LOADING"), view_time, 0)) loading
from tmp8 t1
left join sob as t2 on t1.user_pseudo_id = t2.user_pseudo_id
left join txn as t3 on t1.user_pseudo_id = t3.user_pseudo_id
group by 1,2,3,4
)
select *
--user_pseudo_id, type, session_time, sob
, home_a/session_time spent_at_home
, max_view_time
, session_time/no_screen avg_screen_view
, session_time/no_unique_screen avg_unique_screen_view
, no_screen/no_unique_screen screen_frequency
from tmp9
where 1=1
and session_time > 0
and no_screen > 0
and sob like "%Default 10 VUI_1Jun%"
--"Default 10 VUI_1Jun",
--"outlier_phone",
--"first_lc",
--"Organic bundle",
--"outlier_pseudo",
--"VUI-LONGCHAU",
--"AN-PIZZA",
--"TAP-TAP"
--)

-- select * from tmp8

"""

df = (
    client.query(query_string)
    .result()
    .to_dataframe(
        # Optionally, explicitly request to use the BigQuery Storage API. As of
        # google-cloud-bigquery version 1.26.0 and above, the BigQuery Storage
        # API is used by default.
        create_bqstorage_client=False,
    )
)
# vc = df.article_id.value_counts()
# pairs = {}
# for j,i in enumerate(vc.index.values):
#     #if j%10==0: print(j,', ',end='')
#     USERS = df.loc[df.article_id==i,'customer_id'].unique()
#     vc2 = df.loc[(df.customer_id.isin(USERS))&(df.article_id!=i),'article_id'].value_counts()
#     pairs[i] = vc2
    
# df1 = pd.DataFrame(pairs)
# df1 = df1.fillna(0)
# df2 = pd.DataFrame(vc)
# df1 = df1.join(df2)
# df1.to_csv("cross_brand.csv")